In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# скачиваем кадры из целевого фильма, переделанные в формат _1280*720
#https://drive.google.com/file/d/10hslK8FCgG7Kir7r4pepdI_H6Ya4q2oG/view?usp=drive_link

!pip install -q gdown==4.7.3
!gdown --quiet 10hslK8FCgG7Kir7r4pepdI_H6Ya4q2oG -O image_paths_films_4.zip
!unzip -q "image_paths_films_4.zip"


In [ ]:
#https://drive.google.com/file/d/1F93R9C26iYgeZOaighQr4dfcxj5DHQv4/view?usp=sharing

!pip install -q gdown==4.7.3
!gdown --quiet 1F93R9C26iYgeZOaighQr4dfcxj5DHQv4 -O сад_1280_720.mp4

In [ ]:
# скачиваем датасет с апельсиновыми деревьями
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="CQL5rwm3Xg5OTOzuPH8L")
project = rf.workspace("1-xjgvf").project("orange-8ywhf")
version = project.version(5)
dataset = version.download("yolov8")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
loading Roboflow workspace...
loading Roboflow project...
[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.196`, to intall it `pip install ultralytics==8.0.196`.



Extracting Dataset Version Zip to Orange-5 in yolov8:: 100%|██████████| 10810/10810 [00:16<00:00, 637.68it/s]


In [ ]:
!pip install ultralytics

In [ ]:
!pip install supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

import supervision as sv
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np

# необходим для правильного отображения изображений в колабе
from google.colab.patches import cv2_imshow

Ultralytics YOLOv8.2.45 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.2/78.2 GB disk)


In [ ]:
import yaml

def read_and_print_yaml(file_path):
    # Открываем файл и читаем его содержимое
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)

    # Выводим содержимое файла в нужном формате
    print(f"train: {data['train']}")
    print(f"val: {data['val']}")
    print(f"test: {data['test']}\n")
    print(f"nc: {data['nc']}")
    print(f"names: {data['names']}\n")
    print("roboflow:")
    for key, value in data['roboflow'].items():
        print(f"  {key}: {value}")


In [ ]:
read_and_print_yaml('/content/Orange-5/data.yaml')

train: Orange-5/train/images
val: Orange-5/valid/images
test: ../test/images

nc: 1
names: ['orange']

roboflow:
  license: CC BY 4.0
  project: orange-8ywhf
  url: https://universe.roboflow.com/1-xjgvf/orange-8ywhf/dataset/5
  version: 5
  workspace: 1-xjgvf


In [ ]:
# меняем пути к изображениям на актуальные (необходимо для запуска обучения)
with open('/content/Orange-5/data.yaml', 'r') as file:
    data = yaml.safe_load(file)

# Modify the data
data['train'] = '../train/images'
data['val'] = '../valid/images'

# Save the modified data back to the YAML file
with open('/content/Orange-5/data.yaml', 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

In [ ]:
# проверяем
read_and_print_yaml('/content/Orange-5/data.yaml')

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 1
names: ['orange']

roboflow:
  license: CC BY 4.0
  project: orange-8ywhf
  url: https://universe.roboflow.com/1-xjgvf/orange-8ywhf/dataset/5
  version: 5
  workspace: 1-xjgvf


In [ ]:
# берем изображения из датасета
image_paths = [
    '/content/Orange-5/train/images/00495-3859597343_png.rf.20a1bdc6107f94d07aa3e4e81ee97e86.jpg',
    '/content/Orange-5/train/images/101_orang_jpg.rf.34b2f123e7dd090d20bde1399b44a2b8.jpg',
    '/content/Orange-5/train/images/153_orang_jpg.rf.4fb90a7e02d64a66c47cb7379090e5d0.jpg',
    '/content/Orange-5/train/images/Naranjas_970_png_jpg.rf.fd0997e0e81bf817c4d88d686fa6c21f.jpg'
]


In [ ]:
# берем кадры из нашего целевого видео
image_paths_films4 = [
    '/content/image_paths_films_4/frame120.jpg',
    '/content/image_paths_films_4/frame240.jpg',
    '/content/image_paths_films_4/frame420.jpg',
    '/content/image_paths_films_4/frame600.jpg'
]

<font color=green size=3>
Проверим предположение, что если увеличить размерность кадров из ролика про сад, то возможено детализация повысится и детекция улучшится.
<br>
Попробуем сначала модель обученную на втором "искусственном" датасете.
<br>
</font>

In [ ]:
    # Скачиваем наиболее удачные веса для YOLOv8 модели после обучения на датасете с условиями,похожими на целевой фильм
    yolo_model_best2 = YOLO('/content/ves/best2.pt')


In [ ]:
# проверяем, как предсказывает предобученная nano модель дообученна на 25 эпохах  по кадрам из нашего датасета
for img_in in image_paths:
  results = yolo_model_best2.predict(img_in)
  result = results[0]
  detections = sv.Detections.from_ultralytics(result)

  bounding_box_annotator = sv.BoundingBoxAnnotator()
  label_annotator = sv.LabelAnnotator()

  labels = [
    yolo_model_best2.model.names[class_id]
    for class_id
    in detections.class_id
  ]

  image = cv2.imread(img_in)

  annotated_image = bounding_box_annotator.annotate(
     scene=image, detections=detections)
  annotated_image = label_annotator.annotate(
     scene=annotated_image, detections=detections, labels=labels)

  sv.plot_image(annotated_image)
  for box in result.boxes:
    class_id = result.names[box.cls[0].item()]
    if class_id == 'orange' :
      cords = box.xyxy[0].tolist()
      cords = [round(x) for x in cords]
      conf = round(box.conf[0].item(), 2)
      print("Object type:", class_id)
      print("Coordinates:", cords)
      print("Probability:", conf)
      print("---")


In [ ]:
# проверяем, как предсказывает предобученная nano модель дообученна на 25 эпохах  по кадрам из нашего целевого видео
for img_in in image_paths_films4:
  results = yolo_model_best2.predict(img_in)
  result = results[0]
  detections = sv.Detections.from_ultralytics(result)

  bounding_box_annotator = sv.BoundingBoxAnnotator()
  label_annotator = sv.LabelAnnotator()

  labels = [
    yolo_model_best2.model.names[class_id]
    for class_id
    in detections.class_id
  ]

  image = cv2.imread(img_in)

  annotated_image = bounding_box_annotator.annotate(
     scene=image, detections=detections)
  annotated_image = label_annotator.annotate(
     scene=annotated_image, detections=detections, labels=labels)

  sv.plot_image(annotated_image)
  for box in result.boxes:
    class_id = result.names[box.cls[0].item()]
    if class_id == 'orange' :
      cords = box.xyxy[0].tolist()
      cords = [round(x) for x in cords]
      conf = round(box.conf[0].item(), 2)
      print("Object type:", class_id)
      print("Coordinates:", cords)
      print("Probability:", conf)
      print("---")


<font color=green size=3>
<br>
Теперь попробуем  модель обученную на третьем "малом" датасете.
<br>
</font>

In [ ]:
    # Скачиваем наиболее удачные веса для YOLOv8 модели после обучения на датасете с условиями,похожими на целевой фильм
    yolo_model_best3 = YOLO('/content/ves/best3.pt')


In [ ]:
# проверяем, как предсказывает предобученная nano модель дообученна на 25 эпохах  по кадрам из нашего датасета
for img_in in image_paths:
  results = yolo_model_best3.predict(img_in)
  result = results[0]
  detections = sv.Detections.from_ultralytics(result)

  bounding_box_annotator = sv.BoundingBoxAnnotator()
  label_annotator = sv.LabelAnnotator()

  labels = [
    yolo_model_best3.model.names[class_id]
    for class_id
    in detections.class_id
  ]

  image = cv2.imread(img_in)

  annotated_image = bounding_box_annotator.annotate(
     scene=image, detections=detections)
  annotated_image = label_annotator.annotate(
     scene=annotated_image, detections=detections, labels=labels)

  sv.plot_image(annotated_image)
  for box in result.boxes:
    class_id = result.names[box.cls[0].item()]
    if class_id == 'orange' :
      cords = box.xyxy[0].tolist()
      cords = [round(x) for x in cords]
      conf = round(box.conf[0].item(), 2)
      print("Object type:", class_id)
      print("Coordinates:", cords)
      print("Probability:", conf)
      print("---")


In [ ]:
# проверяем, как предсказывает предобученная nano модель дообученна на 25 эпохах  по кадрам из нашего целевого видео
for img_in in image_paths_films4:
  results = yolo_model_best3.predict(img_in)
  result = results[0]
  detections = sv.Detections.from_ultralytics(result)

  bounding_box_annotator = sv.BoundingBoxAnnotator()
  label_annotator = sv.LabelAnnotator()

  labels = [
    yolo_model_best3.model.names[class_id]
    for class_id
    in detections.class_id
  ]

  image = cv2.imread(img_in)

  annotated_image = bounding_box_annotator.annotate(
     scene=image, detections=detections)
  annotated_image = label_annotator.annotate(
     scene=annotated_image, detections=detections, labels=labels)

  sv.plot_image(annotated_image)
  for box in result.boxes:
    class_id = result.names[box.cls[0].item()]
    if class_id == 'orange' :
      cords = box.xyxy[0].tolist()
      cords = [round(x) for x in cords]
      conf = round(box.conf[0].item(), 2)
      print("Object type:", class_id)
      print("Coordinates:", cords)
      print("Probability:", conf)
      print("---")


<font color=green size=3>
<br>
Теперь попробуем нано модель обученную на первом "основном" датасете .
<br>
</font>

In [ ]:
    # Скачиваем наиболее удачные веса для YOLOv8 модели после обучения на датасете с условиями,похожими на целевой фильм
    yolo_model_best4 = YOLO('/content/ves/best4.pt')


In [ ]:
# проверяем, как предсказывает предобученная nano модель дообученна на 25 эпохах  по кадрам из нашего датасета
for img_in in image_paths:
  results = yolo_model_best4.predict(img_in)
  result = results[0]
  detections = sv.Detections.from_ultralytics(result)

  bounding_box_annotator = sv.BoundingBoxAnnotator()
  label_annotator = sv.LabelAnnotator()

  labels = [
    yolo_model_best4.model.names[class_id]
    for class_id
    in detections.class_id
  ]

  image = cv2.imread(img_in)

  annotated_image = bounding_box_annotator.annotate(
     scene=image, detections=detections)
  annotated_image = label_annotator.annotate(
     scene=annotated_image, detections=detections, labels=labels)

  sv.plot_image(annotated_image)
  for box in result.boxes:
    class_id = result.names[box.cls[0].item()]
    if class_id == 'orange' :
      cords = box.xyxy[0].tolist()
      cords = [round(x) for x in cords]
      conf = round(box.conf[0].item(), 2)
      print("Object type:", class_id)
      print("Coordinates:", cords)
      print("Probability:", conf)
      print("---")


In [ ]:
# проверяем, как предсказывает предобученная nano модель дообученна на 25 эпохах  по кадрам из нашего целевого видео
for img_in in image_paths_films4:
  results = yolo_model_best4.predict(img_in)
  result = results[0]
  detections = sv.Detections.from_ultralytics(result)

  bounding_box_annotator = sv.BoundingBoxAnnotator()
  label_annotator = sv.LabelAnnotator()

  labels = [
    yolo_model_best4.model.names[class_id]
    for class_id
    in detections.class_id
  ]

  image = cv2.imread(img_in)

  annotated_image = bounding_box_annotator.annotate(
     scene=image, detections=detections)
  annotated_image = label_annotator.annotate(
     scene=annotated_image, detections=detections, labels=labels)

  sv.plot_image(annotated_image)
  for box in result.boxes:
    class_id = result.names[box.cls[0].item()]
    if class_id == 'orange' :
      cords = box.xyxy[0].tolist()
      cords = [round(x) for x in cords]
      conf = round(box.conf[0].item(), 2)
      print("Object type:", class_id)
      print("Coordinates:", cords)
      print("Probability:", conf)
      print("---")


<font color=green size=3>
<br>
Видим, что при повышении разрешения детектируемого изображения результат детекции принципиально не поменялся.
<br>
Модель обученная на втором "искуственном" датасете по прежнем не видит апельсинов в садах, хотя при создании датасета активно применялись приемы аугментации.
<br>
Модель обученная на третьем "малом" датасете недостаточно обучена и можно сказать, случайно обнаружила апельсин на одном изображении из первого датасета.
<br>
Модель, даже нано, обученная на первом "основном" датасете отработала хорошо. Для этой модели повышение разрешения детектируемого источника помогло обнаружить больше объектов за счет увеличения их масштаба.
<br>
Ниже попробовал обработать видео. Получилось, но на полный ролик не хватило памяти в колабе (видео стало большим).
<br>

</font>

In [ ]:
results_play = yolo_model_best4.predict(source='/content/сад_1280_720.mp4', save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/614) /content/сад_1280_720.mp4: 352x640 18 oranges, 113.2ms
video 1/1 (frame 2/614) /content/сад_1280_720.mp4: 352x640 18 oranges, 11.1ms
video 1/1 (frame 3/614) /content/сад_1280_720.mp4: 352x640 17 oranges, 10.3ms
video 1/1 (frame 4/614) /content/сад_1280_720.mp4: 352x640 15 oranges, 11.3ms
video 1/1 (frame 5/614) /content/сад_1280_720.mp4: 352x640 17 oranges, 10.7ms
video 1/1 (frame 6/614) /content/сад_1280_720.mp4: 352x640 15 ora

In [ ]:
results_play = yolo_model_best4.predict(source='/content/сад_1280_720.mp4', save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/614) /content/сад_1280_720.mp4: 352x640 18 oranges, 10.5ms
video 1/1 (frame 2/614) /content/сад_1280_720.mp4: 352x640 18 oranges, 10.9ms
video 1/1 (frame 3/614) /content/сад_1280_720.mp4: 352x640 17 oranges, 10.5ms
video 1/1 (frame 4/614) /content/сад_1280_720.mp4: 352x640 15 oranges, 10.6ms
video 1/1 (frame 5/614) /content/сад_1280_720.mp4: 352x640 17 oranges, 10.7ms
video 1/1 (frame 6/614) /content/сад_1280_720.mp4: 352x640 15 oran